In [14]:
import pandas as pd
import datetime
import csv
import numpy as np
import os 
import scipy as sp
import xgboost as xgb
import itertools
import operator
import warnings
warnings.filterwarnings('ignore')

In [15]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
#LabelEncoder` can be used to normalize labels.
from sklearn.base import TransformerMixin
from matplotlib import pylab as plt

plot = True
goal = 'Sales'
myid = 'Id'

In [16]:
#eval metric 评价度量
#定义一些变换和评价准则

def ToWeight(y):
    # y is np.array
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w*(y-yhat)**2))
    return rmspe

def rmspe_xg(yhat,y):
    # y = y.vlaues
    y = y.get_label()  #
    y = np.exp(y) -1
    yhat = np.exp(yhat) -1
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w*(y-yhat)**2))
    return 'rmspe',rmspe


In [17]:
store = pd.read_csv('store.csv')

In [18]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [19]:
train_df = pd.read_csv('train.csv')
train_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [20]:
test_df = pd.read_csv('test.csv')
test_df.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [21]:
def load_data():
    # 加载数据，设定数值型和非数值性数据
    
    store = pd.read_csv('store.csv')
    train_org = pd.read_csv('train.csv',dtype={'StateHoliday':pd.np.string_})
    test_org = pd.read_csv('test.csv',dtype={'StateHoliday':pd.np.string_})
    train = pd.merge(train_org, store, on ='Store', how = 'left')
    test = pd.merge(test_org,store, on='Store', how='left')
    features = test.columns.tolist() 
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    features_numeric = test.select_dtypes(include=numerics).columns.tolist()
    features_non_numeric = [f for f in features if f not in features_numeric]
    return (train,test,features,features_non_numeric)

In [22]:
load_data()[2]


['Id',
 'Store',
 'DayOfWeek',
 'Date',
 'Open',
 'Promo',
 'StateHoliday',
 'SchoolHoliday',
 'StoreType',
 'Assortment',
 'CompetitionDistance',
 'CompetitionOpenSinceMonth',
 'CompetitionOpenSinceYear',
 'Promo2',
 'Promo2SinceWeek',
 'Promo2SinceYear',
 'PromoInterval']

In [23]:
#数据与特征处理
def process_data(train,test,features,features_non_numeric):
    """Feature engineeing and selection"""
    
    # Feature engineering
    train = train[train['Sales'] > 0]
    
    for data in [train, test]:
        data['year'] = data.Date.apply(lambda x: x.split('-')[0])
        data['year'] = data['year'].astype(float)
        data['month'] = data.Date.apply(lambda x: x.split('-')[1])
        data['month'] = data['month'].astype(float)
        data['day'] = data.Date.apply(lambda x: x.split('-')[2])
        data['day'] = data['day'].astype(float)
        
        data['promojan'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Jan" in x else 0)
        #TypeError: argument of type 'float' is not iterable 为什么使用isinstance(x,float)
        data['promofeb'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Feb" in x else 0)
        data['promomar'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Mar" in x else 0)
        data['promoapr'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Apr" in x else 0)
        data['promomay'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "May" in x else 0)
        data['promojun'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Jun" in x else 0)
        data['promojul'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Jul" in x else 0)
        data['promoaug'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Aug" in x else 0)
        data['promosep'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Sep" in x else 0)
        data['promooct'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Oct" in x else 0)
        data['promonov'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Nov" in x else 0)
        data['promodec'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Dec" in x else 0)
        
        
        # Features set
        noisy_features = [myid,'Date']
        features = [c for c in features if c not in noisy_features]
        features_non_numeric = [c for c in features_non_numeric if c not in noisy_features]
        features.extend(['year','month','day'])
        
        # Fill NA
        class DataFrameImputer(TransformerMixin):
            # http://stackoverflow.com/questions/25239958/impute-categorical-missing-values-in-scikit-learn
            def __init__(self):
                """Impute missing values.
                Columns of dtype object are imputed with the most frequent value
                in column.
                Columns of other types are imputed with mean of column.
                """
            def fit(self, X, y=None):
                self.fill = pd.Series([X[c].value_counts().index[0] # mode
                    if X[c].dtype == np.dtype('O') else X[c].mean() for c in X], # mean
                    index=X.columns)
                return self
            def transform(self, X, y=None):
                return X.fillna(self.fill)
            
        train = DataFrameImputer().fit_transform(train)
        test = DataFrameImputer().fit_transform(test)
        
        # Pre-processing non-numberic values
        le = LabelEncoder()
        for col in features_non_numeric:
            le.fit(list(train[col])+list(test[col]))
            train[col] = le.transform(train[col])
            test[col] = le.transform(test[col])
            
        # LR和神经网络这种模型都对输入数据的幅度极度敏感，请先做归一化操作
        scaler = StandardScaler()
        for col in set(features) - set(features_non_numeric) - \
          set([]): # TODO: add what not to scale
            scaler.fit(list(train[col])+list(test[col]))
            train[col] = scaler.transform(train[col])
            test[col] = scaler.transform(test[col])
        return (train,test,features,features_non_numeric)

In [24]:
def XGB_native(train,test,features,features_non_numeric):
    depth = 6
    eta = 0.01
    ntrees = 8000
    mcw = 3
    params = {"objective": "reg:linear",
              "booster": "gbtree",
              "eta": eta,
              "max_depth": depth,
              "min_child_weight": mcw,
              "subsample": 0.7,
              "colsample_bytree": 0.7,
              "silent": 1
              }
    print "Running with params: " + str(params)
    print "Running with ntrees: " + str(ntrees)
    print "Running with features: " + str(features)

    # Train model with local split
    tsize = 0.05
    X_train, X_test = cross_validation.train_test_split(train, test_size=tsize)
    dtrain = xgb.DMatrix(X_train[features], np.log(X_train[goal] + 1))
    dvalid = xgb.DMatrix(X_test[features], np.log(X_test[goal] + 1))
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm = xgb.train(params, dtrain, ntrees, evals=watchlist, early_stopping_rounds=100, feval=rmspe_xg, verbose_eval=True)
    train_probs = gbm.predict(xgb.DMatrix(X_test[features]))
    indices = train_probs < 0
    train_probs[indices] = 0
    error = rmspe(np.exp(train_probs) - 1, X_test[goal].values)
    print error

    # Predict and Export
    test_probs = gbm.predict(xgb.DMatrix(test[features]))
    indices = test_probs < 0
    test_probs[indices] = 0
    submission = pd.DataFrame({myid: test[myid], goal: np.exp(test_probs) - 1})
    if not os.path.exists('result/'):
        os.makedirs('result/')
    submission.to_csv("./result/dat-xgb_d%s_eta%s_ntree%s_mcw%s_tsize%s.csv" % (str(depth),str(eta),str(ntrees),str(mcw),str(tsize)) , index=False)
    # Feature importance
    if plot:
      outfile = open('xgb.fmap', 'w')
      i = 0
      for feat in features:
          outfile.write('{0}\t{1}\tq\n'.format(i, feat))
          i = i + 1
      outfile.close()
      importance = gbm.get_fscore(fmap='xgb.fmap')
      importance = sorted(importance.items(), key=operator.itemgetter(1))
      df = pd.DataFrame(importance, columns=['feature', 'fscore'])
      df['fscore'] = df['fscore'] / df['fscore'].sum()
      # Plotitup
      plt.figure()
      df.plot()
      df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(25, 15))
      plt.title('XGBoost Feature Importance')
      plt.xlabel('relative importance')
      plt.gcf().savefig('Feature_Importance_xgb_d%s_eta%s_ntree%s_mcw%s_tsize%s.png' % (str(depth),str(eta),str(ntrees),str(mcw),str(tsize)))

In [25]:
# print"=> 载入数据中..."
train,test,features,features_non_numeric = load_data()
# print "=> 处理数据与特征工程..."
train,test,features,features_non_numeric = process_data(train,test,features,features_non_numeric)
train.head()

KeyError: 'month'